In [2]:
import os
import pandas as pd
import pickle
import numpy as np

# adding to PATH environment variable
To get librosa and other audio libraries to run. dont know why but jupyter does not load the necessary environment pth variables by itself

In [7]:
path = os.environ.get("PATH")
additional_path = 'C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\mingw-w64\\bin;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\usr\\bin;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\bin;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Scripts;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\bin;C:\\Users\\Michi\\Anaconda3\\condabin'
min_additional_path = "C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\bin;"
combined_path = min_additional_path + path
os.environ["PATH"] = combined_path

os.environ.get("PATH")

'C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\bin;C:\\Users\\Michi\\Anaconda3\\envs\\python_v3-8\\Library\\bin;C:\\Users\\Michi\\Anaconda3;C:\\Users\\Michi\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\Michi\\Anaconda3\\Library\\usr\\bin;C:\\Users\\Michi\\Anaconda3\\Library\\bin;C:\\Users\\Michi\\Anaconda3\\Scripts;C:\\Users\\Michi\\Anaconda3\\condabin;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\;C:\\WINDOWS\\System32\\OpenSSH\\;C:\\MinGW\\bin;C:\\Program Files\\MATLAB\\MATLAB Production Server\\R2015a\\runtime\\win64;C:\\Program Files\\MATLAB\\MATLAB Production Server\\R2015a\\bin;C:\\Program Files\\MATLAB\\MATLAB Production Server\\R2015a\\polyspace\\bin;C:\\Program Files\\dotnet\\;C:\\Program Files\\Git\\cmd;C:\\Users\\Michi\\AppData\\Local\\Programs\\Python\\Python39;C:\\Users\\Michi\\AppData\\Local\\Programs\\Python\\Python39\\Scripts;C:\\Program Files\\CMake\\bin;C:\\Program Files\\WinRAR;C:\\Users\\Mi

In [8]:
import librosa

In [32]:
root_dir = "python"
print(os.getcwd())
_, current_folder = os.path.split(os.getcwd())
if current_folder != root_dir:
    os.chdir("../")
print(os.getcwd())

os.listdir("data/Coswara_processed/")

D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python
D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python


['combined_data.csv',
 'models',
 'pickles',
 'Recordings',
 'reformatted_metadata.csv',
 'testdata.csv']

# Load original meta data file
Make sure that the "combined matadata" file was copied in this directory from the original coswara folder

In [33]:
metadata = pd.read_csv("data/Coswara_processed/combined_data.csv")
metadata.head()

,id,a,covid_status,record_date,ep,g,l_c,l_l,l_s,rU,...,vacc,bd,others_resp,ftg,st,ihd,asthma,others_preexist,cld,pneumonia
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Rename the columns of the Dataframe
The previous names were uninterpretable without additional explanations

In [34]:
meta_data_labels = {
    "id": "user_id",
    "a": "age",
    "covid_status": "covid_health_status",
    "record_date": "record_date",
    "ep": "english_proficiency",
    "g": "gender",
    "l_c": "country",
    "l_l": "local_region",
    "l_s": "state",
    "rU": "returning_user",
    "asthma": "asthma",
    "cough": "cough",
    "smoker": "smoker",
    "test_status": "covid_test_result",
    "ht": "hypertension",
    "cold": "cold",
    "diabetes": "diabetes",
    "diarrhoea": "diarrheoa",
    "um": "was_using_mask",
    "ihd": "ischemic_heart_disease",
    "bd": "breathing_difficulties",
    "st": "sore_throat",
    "fever": "fever",
    "ftg": "fatigue",
    "mp": "muscle_pain",
    "loss_of_smell": "loss_of_smell",
    "cld": "chronic_lung_disease",
    "pneumonia": "pneumonia",
    "ctScan": "has_taken_ct_scan",
    "testType": "type_of_covid_test",
    "test_date": "covid_test_date",
    "vacc": "vaccination_status",  # (y->both doses, p->one dose(partially vaccinated), n->no doses)
    "ctDate": "date_of_ct_scan",
    "ctScore": "ct_score",
    "others_resp": "other_respiratory_illness",
    "others_preexist": "other_preexisting_condition"
}
metadata.rename(meta_data_labels, axis="columns", inplace=True)
metadata.head()

,user_id,age,covid_health_status,record_date,english_proficiency,gender,country,local_region,state,returning_user,...,vaccination_status,breathing_difficulties,other_respiratory_illness,fatigue,sore_throat,ischemic_heart_disease,asthma,other_preexisting_condition,chronic_lung_disease,pneumonia
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Add "recording_error" column to flag invalid recordings
Parameter Error = All values in audio file are 0 resulting in NaN when normalizing

Value Error = length of the audio file = 0 samples

In [35]:
#with open("data/Coswara_processed/pickles/invalid_recordings.pickle", "rb") as f:
#    invalid_recordings = pickle.load(f)
#metadata['recording_error'] = np.nan
#for (error, error_info) in invalid_recordings.items():
#    for ID in error_info["id_list"]:
#        # metadata["recording_error"][metadata["user_id"] == ID] = error
#       metadata.loc[metadata["user_id"] == ID, ["recording_error"]] = error
#print(metadata.loc[:, "recording_error"].value_counts())

# Adding Audio Quality (0|1|2) Columns for each Type of Recording
2(excellent), 1(good), 0(bad)

In [36]:
PATH = "data/Coswara-Data/annotations/"
recording_quality_files = {"audio_quality_deep_breathing":  PATH + "breathing-deep_labels_pravinm.csv",
                           "audio_quality_shallow_breathing": PATH + "breathing-shallow_labels_pravinm.csv",
                           "audio_quality_heavy_cough": PATH + "cough-heavy_labels_debottam.csv",
                           "audio_quality_shallow_cough": PATH + "cough-shallow_labels_debarpan.csv",
                           "audio_quality_counting_fast": PATH + "counting-fast_labels_pravinm.csv",
                           "audio_quality_counting_normal": PATH + "counting-normal_labels_pravinm.csv",
                           "audio_quality_vowel_a": PATH + "vowel-a_labels_debarpan.csv",
                           "audio_quality_vowel_e": PATH + "vowel-e_labels_debottam.csv",
                           "audio_quality_vowel_o": PATH + "vowel-o_labels_updated_neeraj.csv"}

for (feature_name, file_path) in recording_quality_files.items():
    audio_quality_annotations = pd.read_csv(file_path)
    # metadata[feature_name] = np.nan

    rename_dict = {"FILENAME": "user_id",
                   " QUALITY": feature_name}
    audio_quality_annotations.rename(rename_dict, axis="columns", inplace=True)
    audio_quality_annotations.user_id = audio_quality_annotations.user_id.str.split("_").str[0]
    # audio_quality_annotations = audio_quality_annotations.astype({feature_name: int})
    metadata = pd.merge(metadata, audio_quality_annotations, on="user_id", how="outer")

In [37]:
metadata.describe()

,age,ct_score,audio_quality_deep_breathing,audio_quality_shallow_breathing,audio_quality_heavy_cough,audio_quality_shallow_cough,audio_quality_counting_fast,audio_quality_counting_normal,audio_quality_vowel_a,audio_quality_vowel_e,audio_quality_vowel_o
count,2746.000000,153.000000,2746.000000,2746.000000,2233.000000,2232.000000,2233.000000,2233.000000,2233.000000,2746.000000,2232.000000
mean,35.154771,5.307190,1.550255,1.522214,1.800717,1.767025,1.762203,1.758621,1.694133,1.693736,1.554659
std,14.096043,5.369604,0.759533,0.770387,0.533098,0.548358,0.583805,0.579247,0.613503,0.634592,0.693613
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
50%,31.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,43.750000,8.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,99.000000,25.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [38]:
metadata.audio_quality_heavy_cough.value_counts(), metadata.audio_quality_shallow_breathing.value_counts(), 

(2.0    1927
 1.0     167
 0.0     139
 Name: audio_quality_heavy_cough, dtype: int64,
 2    1906
 0     472
 1     368
 Name: audio_quality_shallow_breathing, dtype: int64)

# Create binary values from various subcategories
A participant is allowed to choose only one of the below labels.

['covid_status']=='positive_asymp': Infected with COVID-19 and has no Covid like symptoms

['covid_status']=='positive_mild': Infected with COVID-19 and has mild Covid like symptoms

['covid_status']=='positive_moderate': Infected with COVID-19 and has moderate Covid like symptoms

['covid_status']=='recovered_full': Was infected with COVID-19 and has recovered during data collection.

['covid_status']=='resp_illness_not_identified': Not infected by COVID-19 but has some other respiratory illness

['covid_status']=='no_resp_illness_exposed': Not infected by COVID-19, has no respiratory illness, but has been exposed to SARS-CoV-2 virus. Example, a healthcare worker or somebody staying with a COVID positive patient.

['covid_status']=='healthy': Not infected by COVID-19 and none of the above.

['covid_status']=='under validation ': It is for our internal use related to clinical validation of the Coswara web application.

In [39]:
metadata.covid_health_status.value_counts()

healthy                        1433
positive_mild                   426
no_resp_illness_exposed         248
positive_moderate               165
resp_illness_not_identified     157
recovered_full                  146
positive_asymp                   90
under_validation                 81
Name: covid_health_status, dtype: int64

In [40]:
NEGATIVE_LABELS = ["healthy", "resp_illness_not_identified", "no_resp_illness_exposed"]
POSITIVE_LABELS = ["positive_mild", "positive_moderate", "positive_asymp"]
UNKNOWN_LABELS =  ["under_validation", "recovered_full"]

metadata["covid_positive"] = np.nan
negative_idx = metadata["covid_health_status"].str.contains("|".join(NEGATIVE_LABELS))
positive_idx = metadata["covid_health_status"].str.contains("|".join(POSITIVE_LABELS))
metadata.loc[negative_idx, "covid_positive"] = 0
metadata.loc[positive_idx, "covid_positive"] = 1
metadata["covid_positive"].value_counts()

0.0    1838
1.0     681
Name: covid_positive, dtype: int64

# Finding invalid recordings and adding them into the meta data
Make sure that in the recordings folder, there are only folders with participant IDs (no .DS_store files, no .csv files, only folders)

In [41]:
# import librosa

# def are_recordings_valid(id_participant, recordings_path="data/Coswara_processed/Recordings") -> bool:
# # checks if ALL recordings of a specific participant are valid or not (more than 0 samples, audio that is not just zeros)
# # TODO differentiate between ONE/TWO/THREE files that are invalid compared to ALL recordings that are invalid
#     recording_types = ["cough-heavy", "cough-shallow", "breathing-deep", "breathing-shallow", "counting-fast",
#                        "counting-normal", "vowel-a", "vowel-e", "vowel-o"]
#     path = os.path.join(recordings_path, id_participant)
#     n_valid_recordings = 0

#     for rec_type in recording_types:
#         file_path = f"{os.path.join(path, rec_type)}.wav"
#         try:
#             audio, _ = librosa.load(file_path, sr=None)
#         except FileNotFoundError:
#             return False
#         if len(audio) == 0:            
#             return False
#         elif max(audio) == 0:
#             return False
#     return True

In [42]:
import librosa

def get_recording_metadata(id_participant, recordings_path="data/Coswara_processed/Recordings"):
    # returns original duration and duration after trimming trailing/leading silence for each recording type as a dict
    # returns 0 for both if an error occured/the audio is invalid
    recording_types = ["cough-heavy", "cough-shallow", "breathing-deep", "breathing-shallow", "counting-fast",
                       "counting-normal", "vowel-a", "vowel-e", "vowel-o"]
    path = os.path.join(recordings_path, id_participant)
    duration_original = {}
    duration_trimmed = {}
    
    for rec_type in recording_types:
        file_path = f"{os.path.join(path, rec_type)}.wav"
        try:
            audio, sample_rate = librosa.load(file_path, sr=None)
        except FileNotFoundError:
            duration_original[rec_type], duration_trimmed[rec_type] = 0, 0
        if len(audio) == 0:            
            duration_original[rec_type], duration_trimmed[rec_type] = 0, 0
        elif max(audio) == 0:
            duration_original[rec_type], duration_trimmed[rec_type] = 0, 0
        else:
            duration_original[rec_type] = len(audio)/sample_rate
            audio, _ = librosa.effects.trim(audio, top_db=54)
            duration_trimmed[rec_type] = len(audio)/sample_rate
            
        
    return duration_original, duration_trimmed

OSError: cannot load library 'libsndfile.dll': error 0x7e

In [11]:
import random
recordings_path = "data/Coswara_processed/Recordings"
participant_ids = os.listdir(recordings_path)
participant_id = random.choice(participant_ids)
# participant_id = "0dE8pniDGCZp0gYMy23fvjpZBDr1"
get_recording_metadata(participant_id, recordings_path=recordings_path)
#are_recordings_valid(participant_id)

NameError: name 'get_recording_metadata' is not defined

In [167]:
# this may take a while executing
# maybe save the result and make an if statement (if there is no saved file yet(?) execute this, otherwise load from memory)
# recordings_valid = [are_recordings_valid(participant) for participant in participant_ids]

try:
    is_recording_valid_df = pd.read_csv("data/Coswara_processed/pickles/is_recording_valid_df.csv")
    print("loaded valid/invalid recordings dataframe from disk!")
except FileNotFoundError:  
    recordings_valid = []
    for idx, participant in enumerate(participant_ids):
        print(f"{idx} / {len(participant_ids)}")
        recordings_valid.append(are_recordings_valid(participant))
    is_recording_valid_df = pd.DataFrame({"ids":participant_ids, "valid_recording":recordings_valid})
    is_recording_valid_df.to_csv("data/Coswara_processed/pickles/is_recording_valid_df.csv")
    
is_recording_valid_df.valid_recording.value_counts()

loaded valid recordings from disk!


True     2580
False     166
Name: valid_recording, dtype: int64

In [31]:
from IPython.display import clear_output

In [44]:
recording_types = ["cough-heavy", "cough-shallow", "breathing-deep", "breathing-shallow", "counting-fast",
                   "counting-normal", "vowel-a", "vowel-e", "vowel-o"]
try:
    audio_recording_metadata = pd.read_csv("data/Coswara_processed/pickles/duration_df.csv")
    print("loaded audio_recording_metadata dataframe from disk!")
except FileNotFoundError:  
    durations_original, durations_trimmed = [], []
    for idx, participant in enumerate(participant_ids):
        clear_output(wait=True)
        print(f"{idx+1} / {len(participant_ids)}")
        original, trimmed = get_recording_metadata(participant)
        durations_original.append(original), durations_trimmed.append(trimmed)
   
    
    all_durations_orig = {}
    all_durations_trim = {}
    for rec_type in recording_types:
        all_durations_orig[f"duration_original_{rec_type}"] = [round(recording[rec_type],3) for recording in durations_original]
        all_durations_trim[f"duration_trimmed_{rec_type}"] = [round(recording[rec_type],3) for recording in durations_trimmed]

    duration_dict = {"ids":participant_ids}
    duration_dict.update(all_durations_orig)
    duration_dict.update(all_durations_trim)
    audio_recording_metadata = pd.DataFrame(duration_dict)

    duration_df.to_csv("data/Coswara_processed/pickles/duration_df.csv")
#all_durations_orig, all_durations_trim

loaded audio_recording_metadata dataframe from disk!
